In [31]:
import numpy as np
from PIL import Image
import io
from numpy.dtypes import StrDType
from matplotlib import pyplot as plt
from package.utils.logger import logger
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scipy.stats import multivariate_normal


In [30]:

# Bootstrap
raw_dataset = np.load('.ds.tiny/dataset.zip')

dataset = {
    'train': {
        'data': [],
        'names': [],
        'labels': [],
        'unique_labels': [],
    },
    'valid': {
        'data': [],
        'names': [],
        'labels': [],
        'unique_labels': [],
    }
}

images_resize_shape = (128,128)

# For each image we have the path from which we extract the name and the label of the image
for dsKey in raw_dataset.keys():
    splittedKey = dsKey.split('/')

    img_type = splittedKey[2]
    img_label = splittedKey[3]
    img_name = splittedKey[4]
    
    img = Image.open(io.BytesIO(raw_dataset[dsKey])).resize(images_resize_shape, Image.Resampling.LANCZOS)
    img_array = np.asarray(img).reshape(images_resize_shape[0]*images_resize_shape[1], 3)
    
    dataset[img_type]['data'].append(img_array)
    dataset[img_type]['names'].append(img_name)
    dataset[img_type]['labels'].append(img_label)

for img_type in dataset.keys():
    dataset[img_type]['data'] = np.asarray(dataset[img_type]['data'])
    dataset[img_type]['names'] = np.asarray(dataset[img_type]['names'])

    dataset[img_type]['unique_labels'], dataset[img_type]['labels'] = np.unique(np.asarray(dataset[img_type]['labels']), return_inverse=True)

    logger.info([f'data shape({img_type})', dataset[img_type]['data'].shape])
    logger.info([f'data labels({img_type})', dataset[img_type]['labels'].shape])
    logger.info([f'data unique labels({img_type})', dataset[img_type]['unique_labels']])


DEFAULT_LOGGER: 2024-06-03 17:20:30,308 | INFO | 2263878544.py:42 ['data shape(train)', (1500, 16384, 3)]
DEFAULT_LOGGER: 2024-06-03 17:20:30,309 | INFO | 2263878544.py:43 ['data labels(train)', (1500,)]
DEFAULT_LOGGER: 2024-06-03 17:20:30,309 | INFO | 2263878544.py:44 ['data unique labels(train)', array(['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel',
       'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu'],
      dtype='<U12')]
DEFAULT_LOGGER: 2024-06-03 17:20:30,312 | INFO | 2263878544.py:42 ['data shape(valid)', (500, 16384, 3)]
DEFAULT_LOGGER: 2024-06-03 17:20:30,313 | INFO | 2263878544.py:43 ['data labels(valid)', (500,)]
DEFAULT_LOGGER: 2024-06-03 17:20:30,313 | INFO | 2263878544.py:44 ['data unique labels(valid)', array(['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel',
       'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu'],
      dtype='<U12')]


In [10]:

# Dimensionality reduction 



train_images = train_images[:,:,:,:].reshape((train_images.shape[0], train_images.shape[1]*train_images.shape[2], 3))
test_images = test_images[:,:,:,:].reshape((test_images.shape[0], test_images.shape[1]*test_images.shape[2], 3))

pca = [PCA(n_components=10), PCA(n_components=10), PCA(n_components=10)]

pca_train_images = []
pca_test_images = []

for i in range(3):
    pca[i].fit(train_images[:,:,i]) 
    pca_train_images.append(pca[i].transform(train_images[:,:,i])) 
    pca_test_images.append(pca[i].transform(test_images[:,:,i])) 

    logger.info([f'PCA{i} variance ratio: ', np.sum(pca[i].explained_variance_ratio_)])

NameError: name 'train_images' is not defined

In [45]:
estimators = []

for labelIdx in range(unique_train_labels.shape[0]):
    estimators.append([])
    label = unique_train_labels[labelIdx]

    for i in range(3):
        estimators[labelIdx].append(multivariate_normal(np.mean(pca_train_images[i][train_labels == label], axis=0), np.cov(np.transpose(pca_train_images[i][train_labels == label])), allow_singular=True))

In [48]:
estims = np.zeros((len(estimators), 3, test_images.shape[0]))

for estimator_idx in range(len(estimators)):
    for color_idx in range(3):
        estims[estimator_idx,color_idx] = estimators[estimator_idx][color_idx].pdf(pca_test_images[color_idx])

max_estims_idx = np.argmax(estims.sum(1),0)

max_estims = unique_test_labels[max_estims_idx]

print(np.count_nonzero(max_estims == test_labels)/test_images.shape[0])

plt.show()
ConfusionMatrixDisplay(confusion_matrix(test_labels, max_estims),display_labels=unique_test_labels).plot()


4.327427937216965e-37
2.0696532111436884e-37
1.5736377499224094e-37
0.248
